In [1]:
import torch
import gymnasium    
import numpy as np
from differentiable_representation import NystromFeatureExtractor

In [2]:
env = gymnasium.make('Pendulum-v1')
feature_extractor = NystromFeatureExtractor(env)

In [3]:
def pendulum_3d(obs, action, g=10.0, m=1., l=1., max_a=2., max_speed=8., dt=0.05):
    th = torch.atan2(obs[:, 1], obs[:, 0])  # 1 is sin, 0 is cosine
    thdot = obs[:, 2]
    action = torch.reshape(action, (action.shape[0],))
    u = torch.clip(action, -max_a, max_a)
    newthdot = thdot + (3. * g / (2 * l) * torch.sin(th) + 3.0 / (m * l ** 2) * u) * dt
    newthdot = torch.clip(newthdot, -max_speed, max_speed)
    newth = th + newthdot * dt
    new_states = torch.empty((obs.shape[0], 3))
    new_states[:, 0] = torch.cos(newth)
    new_states[:, 1] = torch.sin(newth)
    new_states[:, 2] = newthdot
    return new_states
feature_extractor.set_dynamics(pendulum_3d)

In [4]:
env = gymnasium.make('Pendulum-v1')
feature_extractor = NystromFeatureExtractor(env)
feature_extractor.set_dynamics(pendulum_3d)
sprime, fsa = feature_extractor.random_sample_sprime_fsa()

In [12]:
K1 = feature_extractor.get_kernel_matrix_asymmetric(sprime, fsa)
K2 = feature_extractor.kernel_matrix(torch.from_numpy(sprime), torch.from_numpy(fsa))
torch.equal(torch.from_numpy(K1), K2)

False

In [30]:
m = sprime.shape[0]
n = fsa.shape[0]
dx1 = np.empty((m, n, 3))
for i in np.arange(m):
    for j in np.arange(n):
        dx1[i, j] = sprime[i, :] - fsa[j, :]
dx2 = torch.from_numpy(sprime).unsqueeze(1) - torch.from_numpy(fsa).unsqueeze(0)
torch.equal(torch.from_numpy(dx1), dx2)

True

In [29]:
dx2 = np.expand_dims(sprime, axis=1) - np.expand_dims(fsa, axis=0)
np.all(np.equal(dx1, dx2))

True

In [20]:
torch.from_numpy(fsa).unsqueeze(0).size()

torch.Size([1, 512, 3])

### Get nystrom features by SVD for simulator

False